In [1]:

import pandas as pd

In [50]:

Title1 = "Don't abandon chimpanzees for whom you promised to provide lifetime care!"

Text1 = " Sixty six captive chimpanzees in Liberia, Africa have been abandoned by the \
    New York Blood Center (NYBC)—an organization with assets of $450 million and major \
    corporate partners. The chimpanzees are in danger of dehydration and starvation. Please\
    sign the petition to urge NYBC to reinstate funding for this chimpanzee colony before its,"

Image1 = 0
Tweet1 = 1
Goal_No1 = '20000'

raw_dict={'Title':[Title1],'Text':[Text1],'Goal_No':[Goal_No1] }

In [51]:
df=pd.DataFrame(raw_dict)


In [52]:
df

,Goal_No,Text,Title
0,20000,"Sixty six captive chimpanzees in Liberia, Afr...",Don't abandon chimpanzees for whom you promise...


In [53]:
def str2float(s):
    import re
    s = str(s).strip().replace(',','')
    s = re.sub('[^0-9]+', '0', s)
    return float(s)

In [64]:
def data_clean(Title1,Text1,Goal_No1):
    import re
    def str2float(s):
        s = str(s).strip().replace(',','')
        s = re.sub('[^0-9]+', '0', s)
        return float(s)
    
    def remove_sw(words, sw):
        word = [w for w in words if not w in sw]
        return word
    
    def clean_text( col):
        letters_only=(col.apply(lambda x:re.sub(u"\xa0",u" ",x))
        #.apply(lambda x: BeautifulSoup(x).get_text())
        .apply(lambda x:re.sub("[^a-zA-Z]"," ",x))
                 )
        lower_case=letters_only.apply(lambda x: x.lower().split())
        from nltk.corpus import stopwords # Import the stop word list
        stopwords=set(stopwords.words("english"))
        clean_texts = []
        num_texts = col.size
        for i in range( 0, num_texts ):
        # Call our function for each one, and add the result to the list of
        # clean 
            clean_texts.append( " ".join(remove_sw(lower_case[i],stopwords)))
        return clean_texts
    

    raw_dict={'Title':[Title1],'Text':[Text1],'Goal_No':[Goal_No1]}
    df=pd.DataFrame(raw_dict)
    df.Goal_No=df.Goal_No.apply(str2float)
    df.Text=clean_text(df.Text)
    df.Title=clean_text(df.Title)
    df['Text_len']=df.Text.str.split(' ').apply(lambda x: len(x))
    df['Title_len']=df.Title.str.split(' ').apply(lambda x: len(x))
    df['Text_str_len']=df.Text.str.split(' ').apply(lambda x: sum(len(w) for w in x)/ len(x))
    df['Title_str_len']=df.Title.str.split(' ').apply(lambda x: sum(len(w) for w in x)/ len(x))
    df['Text_len_p1']=(df['Text_len']<270)*df['Text_len']
    df['Text_len_p2']=(df['Text_len']>270)*df['Text_len']
    return df
    
    
    


In [65]:
df=data_clean(Title1,Text1,Goal_No1)

In [68]:
def data_vectorize(df):
    import pickle
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    from sklearn.feature_extraction.text import CountVectorizer
    import numpy as np
    
    with open('text_court_vectorizer.pickle','rb') as f:
        text_court_vectorizer=pickle.load(f)
    with open('title_court_vectorizer.pickle','rb') as f:
        title_court_vectorizer=pickle.load(f)
    with open('total_scale_train.pickle','rb') as f:
        total_scale_train=pickle.load(f)
    with open('train_pca.pickle','rb') as f:
        train_pca=pickle.load(f)
    
    
    test_text_features = text_court_vectorizer.transform(df.Text)
    test_text_features = test_text_features.toarray()  
    test_title_features = title_court_vectorizer.transform(df.Title)
    test_title_features = test_title_features.toarray()
    

    test_all_features=total_scale_train.transform(np.concatenate((
                        test_text_features,test_title_features,
                        df[['Title_str_len', 'Text_len_p1', 'Text_len_p2', 
                        'Title_len', 'Text_str_len']]),axis=1))
    
    
    x_test_pca=train_pca.transform(test_all_features)
    
    return x_test_pca
    
    
    

In [69]:
x_test_pca=data_vectorize(df)

In [72]:
def data_predict(x_test_pca):
    with open('train_Elastic.pickle','rb') as f:
        train_Elastic=pickle.load(f)
    test_predict=train_Elastic.predict(x_test_pca[:,0:500])
    return test_predict

In [87]:
y_predict=data_predict(x_test_pca)

In [74]:
int(10**data_predict(x_test_pca))

3975

In [75]:
def plot_prob(mu,log_goal):
    import seaborn as sns
    import matplotlib.pyplot as plt
    import numpy as np
    import matplotlib.mlab as mlab
    import math
    variance=0.88
    x = np.linspace(mu-4*variance,mu+4*variance, 100)
    y = mlab.normpdf(x, mu, variance ** (1/2))
    plot = plt.plot(x,y)
    vertical_line = log_goal
    plt.fill_betweenx(y, vertical_line, x, where=x<vertical_line, alpha=0.5)
    


In [132]:
def plotly_prob(mu, log_goal):
    
    variance=0.88
    x= np.linspace(mu-5*variance,mu+5*variance, 100)
    x_new=[10**w for w in x ]
    y = mlab.normpdf(x, mu, (variance ** (1/2)))

    x1= np.linspace(log_goal, mu+5*variance, 100)
    x1_new=[10**w for w in x1 ]
    y1 = mlab.normpdf(x1, mu, (variance ** (1/2)))
    
    x2=[10**log_goal for _ in range(100)]
    y2= np.linspace(0, mlab.normpdf(log_goal, mu, variance ** (1/2)), 100)
    
    
    py.sign_in(username='cherry8177', api_key='3FTZrZRMQSDYshs8rxJ7')
    trace1 = {
      "x": x_new ,
      "y": y,

    "name": "PDF of Prediction Number", 
      "type": "scatter"
    }
    trace2 = {
      "x": x1_new,
      "y": y1,
      "fill": "tozeroy", 
      "name": "Prob(Predction> Your Goal)", 
      "type": "scatter"
    }
    trace3 = {
      "x": x2,
      "y": y2,
      
      "name": "Your Goal", 
      "type": "scatter"
    }

    layout = go.Layout(
        title='Your Chace of Reaching the Goal Number',
        xaxis=dict(
            title='Supporter Number',
            type='log',
        autorange=True,
            titlefont=dict(
                family='Courier New, monospace',
                size=24,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Probability',
            titlefont=dict(
                family='Courier New, monospace',
                size=24,
                color='#7f7f7f'
            )
        )
    )

    data = go.Data([trace1, trace2,trace3])

    fig = go.Figure(data=data, layout=layout)
    #div = plotly.offline.plot(fig, show_link=False, output_type="div", include_plotlyjs=False)

    #py.iplot(fig)
    return fig

In [133]:
py.iplot(plotly_prob(y_predict,4))

In [81]:
def get_prob(goal_No, y_predict):
    import scipy.stats
    prob=scipy.stats.norm(goal_No,0.88**0.5).cdf(y_predict)
    return prob[0]